# 卷积神经网络 -- 从0 开始 

In [1]:
from mxnet import nd 
# 输入输出数据格式是 batch x channel x height x width ,这里batch和channel 都是1
# 权重格式是 input_filter x output_filter x height x width ,这里的input_filter和out_filter 

w=nd.arange(4).reshape((1,1,2,2))
b=nd.array([1]) 
data=nd.arange(9).reshape((1,1,3,3)) 
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[1]) 
print("input:",data,"\n\n weight:",w,"\n\nbias",b ,"\n\noutput:",out) 

input: 
[[[[0. 1. 2.]
   [3. 4. 5.]
   [6. 7. 8.]]]]
<NDArray 1x1x3x3 @cpu(0)> 

 weight: 
[[[[0. 1.]
   [2. 3.]]]]
<NDArray 1x1x2x2 @cpu(0)> 

bias 
[1.]
<NDArray 1 @cpu(0)> 

output: 
[[[[20. 26.]
   [38. 44.]]]]
<NDArray 1x1x2x2 @cpu(0)>


In [2]:
w=nd.arange(8).reshape((1,2,2,2))
data=nd.arange(18).reshape((1,2,3,3)) 
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[0]) 
print("input:",data,"\n\n weight:",w,"\n\nbias:",b,"\n\noutput:",out) 

input: 
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 

 weight: 
[[[[0. 1.]
   [2. 3.]]

  [[4. 5.]
   [6. 7.]]]]
<NDArray 1x2x2x2 @cpu(0)> 

bias: 
[1.]
<NDArray 1 @cpu(0)> 

output: 
[[[[269. 297.]
   [353. 381.]]]]
<NDArray 1x1x2x2 @cpu(0)>


In [3]:
w=nd.arange(16).reshape((2,2,2,2)) 
data=nd.arange(18).reshape((1,2,3,3))
b=nd.array([1,2]) 

out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[0])
print("input:",data,"\n\nweight:",w,"\n\n bias:",b,"\n\noutput:",out) 


input: 
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 

weight: 
[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 4.  5.]
   [ 6.  7.]]]


 [[[ 8.  9.]
   [10. 11.]]

  [[12. 13.]
   [14. 15.]]]]
<NDArray 2x2x2x2 @cpu(0)> 

 bias: 
[1. 2.]
<NDArray 2 @cpu(0)> 

output: 
[[[[ 269.  297.]
   [ 353.  381.]]

  [[ 686.  778.]
   [ 962. 1054.]]]]
<NDArray 1x2x2x2 @cpu(0)>


# 池化层 （pooling) 
因为卷积层每次作用在一个窗口，它对位置很敏感。池化层能够很好的缓解这个问题。它跟卷积类似每次看一个小窗口，然后选择窗口里面最大的元素，或者平均元素作为输出。 

In [4]:
data=nd.arange(18).reshape((1,2,3,3,)) 
max_pool=nd.Pooling(data=data,pool_type='max',kernel=(2,2))
avg_pool=nd.Pooling(data=data,pool_type='avg',kernel=(2,2)) 
print('data:',data,'\n\nmax pooling:', max_pool,'\n\n avg pooling:',avg_pool) 

data: 
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 

max pooling: 
[[[[ 4.  5.]
   [ 7.  8.]]

  [[13. 14.]
   [16. 17.]]]]
<NDArray 1x2x2x2 @cpu(0)> 

 avg pooling: 
[[[[ 2.  3.]
   [ 5.  6.]]

  [[11. 12.]
   [14. 15.]]]]
<NDArray 1x2x2x2 @cpu(0)>


# 获取数据 
我们继续使用FashionMNIST (希望你还没有彻底厌烦这个数据） 

In [5]:
import sys  
import time 
from mxnet import autograd as autograd  
sys.path.append('..')   
import utils
batch_size=256 
train_data,test_data = utils.load_data_fashion_mnist(batch_size) 

In [6]:
type(train_data) 

utils.DataLoader

In [7]:
type(test_data)

utils.DataLoader

In [8]:
for data,label in train_data:
    print(type(data))
    print(type(label))
    print(data.shape)
    print(label.shape)
    break

<class 'mxnet.ndarray.ndarray.NDArray'>
<class 'mxnet.ndarray.ndarray.NDArray'>
(256, 1, 28, 28)
(256,)


# 定义模型
因为卷积网络计算比全连接要复杂，这里我们默认使用GPU来计算。如果GPU不能用，默认使用CPU 

In [9]:
import mxnet as mx 
try:
    ctx=mx.gpu(1)
    __=nd.zeros((1,),ctx=ctx)
except:
    ctx=mx.cpu()
ctx 

cpu(0)

In [10]:
weight_scale=0.01 
num_outputs=10 
num_fc=128 
# output channels=20 ,kernel = (5,5) 
W1=nd.random_normal(shape=(20,1,5,5),scale=weight_scale,ctx=ctx)
b1=nd.zeros(W1.shape[0],ctx=ctx) 

# output channels=50 ,kernel = (3,3) 
W2=nd.random_normal(shape=(50,20,3,3),scale=weight_scale,ctx=ctx) 
b2=nd.zeros(W2.shape[0],ctx=ctx)

# output dim = 128 
W3=nd.random_normal(shape=(1250,128),scale=weight_scale,ctx=ctx) 
b3=nd.zeros(W3.shape[1],ctx=ctx) 

# output dim =10 
W4=nd.random_normal(shape=(W3.shape[1],10),scale=weight_scale,ctx=ctx)
b4=nd.zeros(W4.shape[1],ctx=ctx) 

params=[W1,b1,W2,b2,W3,b3,W4,b4] 
for param in params:
    param.attach_grad()

卷积模块通常是" 卷积层-激活层-池化层".然后转成2D 矩阵输出给后面的全连接层.

In [11]:
def net(X,verbose=False):
    X=X.as_in_context(W1.context) 
    # 第一层卷积 
    h1_conv=nd.Convolution(
                data=X,weight=W1,bias=b1,kernel=W1.shape[2:],num_filter=W1.shape[0])
    h1_activation=nd.relu(h1_conv) 
    h1=nd.Pooling( 
         data=h1_activation,pool_type='max',kernel=(2,2),stride=(2,2))
    # 第二层卷积 
    h2_conv=nd.Convolution( 
        data=h1,weight=W2,bias=b2,kernel=W2.shape[2:],num_filter=W2.shape[0])
    h2_activation=nd.relu(h2_conv) 
    h2=nd.Pooling(data=h2_activation,pool_type='max',kernel=(2,2),stride=(2,2))
    h2=nd.flatten(h2)
    # 第一层全连接 
    h3_linear=nd.dot(h2,W3)+b3 
    h3=nd.relu(h3_linear) 
    # 第二次全连接
    h4_linear=nd.dot(h3,W4)+b4 
    if verbose:
        print('lst conv block:',h1.shape)
        print('2nd conv block:',h2.shape)
        print('1st dense:',h3.shape)
        print('2nd dense:',h4_linear.shape) 
        print('output:',h4_linear) 
    return h4_linear 

测试一下，输出中间结果形状（当然可以直接打印结果）和最终结果 

In [12]:

for data,__ in train_data:
    print(data.shape)
    net(data,verbose=True) 
    break 

(256, 1, 28, 28)
lst conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[ 3.47410460e-05 -6.55693293e-05  1.78395814e-04 ...  5.81464265e-05
  -3.07796472e-05  8.20601854e-05]
 [ 7.23581979e-05 -2.07145858e-05  1.77788694e-04 ...  8.71310185e-05
  -2.49837412e-05  1.26747342e-04]
 [ 8.03927251e-05 -6.17858095e-05  1.08732209e-04 ...  1.14658505e-04
  -5.28171986e-05  8.26782125e-05]
 ...
 [ 1.31961569e-04 -3.13651617e-05  1.74727582e-04 ...  7.72152343e-05
  -1.30071159e-04  1.35919536e-05]
 [ 1.68138959e-05  9.07204321e-06  7.59457325e-05 ...  5.42345842e-05
  -4.99772068e-05  2.42436854e-05]
 [ 7.24911952e-05  8.52160554e-07  1.78394519e-04 ...  6.82041791e-05
  -1.14313967e-04 -3.64129701e-05]]
<NDArray 256x10 @cpu(0)>


# 训练 
跟前面没有什么不同 


In [13]:
def SGD(params,lr):
    for param in params:
        param[:]=param -lr * param.grad 
        
def accuracy(output,label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

In [14]:
from mxnet import gluon 
from time import * 
begin_time=time() 
softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss()
learning_rate=0.2
for epoch in range(5):
    train_loss=0. 
    train_acc=0. 
    for data,label in train_data:
        label=label.as_in_context(ctx)
        with autograd.record():
            output=net(data)
            loss=softmax_cross_entropy(output,label) 
        loss.backward()
        utils.SGD(params,learning_rate/batch_size) 
        train_loss+=nd.mean(loss).asscalar()
        train_acc+=utils.accuracy(output, label.astype("float32"))
    test_acc=utils.evaluate_accuracy(test_data,net,ctx)
    print("Epoch %d . loss :%f ,train acc %f,Test acc%f "% 
          (epoch,train_loss/len(train_data),train_acc/len(train_data),test_acc)
          )  
       

end_time=time()
print(end_time-begin_time)
        

Epoch 0 . loss :2.299468 ,train acc 0.119391,Test acc0.270333 
Epoch 1 . loss :1.709757 ,train acc 0.361128,Test acc0.695413 
Epoch 2 . loss :0.639784 ,train acc 0.751769,Test acc0.791366 
Epoch 3 . loss :0.526086 ,train acc 0.795656,Test acc0.810096 
Epoch 4 . loss :0.474921 ,train acc 0.817625,Test acc0.832131 
34.827566385269165
